In [1]:
from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset_baby/'
sequence = '00'
image_instances_path = 'pipeline_baby/vfm-labels/sam/00/'
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [2]:
from services.preprocessing.common.config import ConfigDTO

config = ConfigDTO(**{
    'dataset': kitti,
    'start_index': 20,
    'end_index': 23,
    'start_image_index_offset': 3,
    'cam_name': 'cam2',
    'R': 15,
    'nb_neighbors': 25,
    'std_ratio': 5.0,
    'voxel_size': 0.3
})

In [3]:
from services.preprocessing.init.map import InitMapProcessor
from services.preprocessing.init.instances_matrix import InitInstancesMatrixProcessor

init_pcd = InitMapProcessor().process(config)
points2instances = InitInstancesMatrixProcessor().process(config, init_pcd)

In [4]:
from services.preprocessing.not_zero import SelectionNotZeroProcessor
from services.preprocessing.in_cube import SelectionInCubeProcessor
from services.preprocessing.statistical_outlier import StatisticalOutlierProcessor

processors = [
    SelectionNotZeroProcessor(),
    SelectionInCubeProcessor(),
    StatisticalOutlierProcessor()
]

In [5]:
import copy

pcd = copy.deepcopy(init_pcd)
for processor in processors:
    pcd, points2instances = processor.process(config, pcd, points2instances)

pcd_for_clustering = copy.deepcopy(pcd)

In [6]:
from services.preprocessing.voxel_down import VoxelDownProcessor

pcd, points2instances, voxel_results = VoxelDownProcessor().process(config, pcd, points2instances)
trace = voxel_results[2]

In [7]:
import numpy as np

from scipy.spatial.distance import cdist
from utils.distances_utils import sam_label_distance

points = np.asarray(pcd.points)
spatial_distance = cdist(points, points)

dist, masks = sam_label_distance(points2instances, spatial_distance, 2, 10)

In [8]:
from utils.distances_utils import remove_isolated_points

dist, points, trace = remove_isolated_points(dist, points, trace)

In [9]:
dist.shape

(2294, 2294)

In [10]:
from services.normalized_cut_service import normalized_cut

T = 0.8
eigenval = 2
clusters = normalized_cut(dist, np.array([i for i in range(len(points))], dtype=int), T, eigenval)

mcut = 0.07493336030744618
mcut = 2.505891488481912e-07
mcut = 0.9375
mcut = 3.1759498681164605e-05
mcut = 0.011324370647003324
mcut = 0.017651533877362315
mcut = 0.10908327126008319
mcut = 6.864119815108579e-10
mcut = 8.098172204952483e-07
mcut = 0.1720543191105628
mcut = 0.9696840288009956
mcut = 0.2853685026907551
mcut = 0.0010549518154947823
mcut = 0.022031056379760407
mcut = 0.05196230279514451
mcut = 0.6705513323317656
mcut = 1.6529056778151462e-07
mcut = 0.1698842153857757
mcut = 0.842809364548495
mcut = 0.5909090909090909
mcut = inf
mcut = 0.7072474386415846
mcut = 0.12740358841927588
mcut = 0.2283826777907227
mcut = 0.9171150781737143
mcut = 0.9524266491534603
mcut = 0.14427115409276933
mcut = 0.9232535732994087
mcut = 0.8257923347806763


/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 4 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/proj/lidar-labelling/services/normalized_cut_service.py:27: RuntimeWarning: invalid value encountered in scalar divide
  return (cost / assoc_a) + (cost / assoc_b)
/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 11 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 8 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 3 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


mcut = 0.4524553293426566
mcut = 0.8531266906542778
mcut = 0.9649873001786087
mcut = 0.6234653714732188
mcut = 0.1417824976351775
mcut = 0.9417114706476364
mcut = 0.9577528107020559
mcut = inf
mcut = 0.9717783039850953
mcut = 0.43450935239530714
mcut = 6.913700000114298e-05
mcut = 0.005313751138730721
mcut = inf
mcut = 9.860873286828943e-09
mcut = 1.4510205010154765e-07
mcut = 0.9473684210526315
mcut = 8.044061353264054e-07
mcut = 0.9090909090909091
mcut = 3.891303828172882e-06
mcut = 1.0525851315517228e-05
mcut = 0.9638023793835037
mcut = 0.006813803681831905
mcut = 0.021157665205122075
mcut = 0.7649975538930288
mcut = 0.8999999999999999
mcut = inf
mcut = 0.08862924466671161
mcut = 0.05504898410729998
mcut = 0.8703923205298658
mcut = 0.07110769022347543
mcut = 0.5149258804684531
mcut = 0.34920753337983124
mcut = 0.8822134387351779
mcut = 0.5453377740323405
mcut = 0.5828027462589546
mcut = 0.5909090909090909
mcut = inf
mcut = inf
mcut = 0.18277469871607704
mcut = 0.9009510597341176
mcu

/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 18 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 279 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


In [11]:
len(clusters)

65

In [12]:
from utils.pcd_utils import color_pcd_by_clusters_and_voxels

pcd_clustered = color_pcd_by_clusters_and_voxels(pcd_for_clustering, trace, clusters)

In [13]:
import open3d as o3d

o3d.visualization.draw_geometries([pcd_clustered])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [15]:
executed_id = 1
name = "labeled_cloud/clouds_{}_{}_R{}_voxel{}_T{}_eigenval{}_execute_id{}".format(start_index, end_index, R, voxel_size, T, eigenval, executed_id)
filename = name.replace(".", "l") + ".pcd"

o3d.io.write_point_cloud(filename, pcd_clustered)

True